2576183S

# RecSys M 2021 - Exercise 3 Template

The aims of this exercise are:
 - Explore a different recommendation dataset
 - Develop and evaluate baseline recommender systems
 - Implement hybrid recommender models
 - Explore diversification issues in recommender systems
 - Revise other material from the lectures.

As usual, there is a corresponding Quiz on Moodle for this Exercise, which should be answered as you proceed. For more details, see the Exercise 3 specification.



# Part-Pre. Preparation 

## Pre 1. Setup Block

This exercise will use the [Goodreads]() dataset for books. These blocks setup the data files, Python etc.

In [ ]:
!rm -rf ratings* books* to_read* test*

!curl -o ratings.csv "http://www.dcs.gla.ac.uk/~craigm/recsysH/coursework/final-ratings.csv" 
!curl -o books.csv "http://www.dcs.gla.ac.uk/~craigm/recsysH/coursework/final-books.csv"
!curl -o to_read.csv "http://www.dcs.gla.ac.uk/~craigm/recsysH/coursework/final-to_read.csv"
!curl -o test.csv "http://www.dcs.gla.ac.uk/~craigm/recsysH/coursework/final-test.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7631k  100 7631k    0     0  4805k      0  0:00:01  0:00:01 --:--:-- 4802k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2366k  100 2366k    0     0  2558k      0 --:--:-- --:--:-- --:--:-- 2555k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7581k  100 7581k    0     0  7705k      0 --:--:-- --:--:-- --:--:-- 7697k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1895k  100 1895k    0     0  2366k      0 --:--:-- --:--:-- --:--:-- 2363k


In [ ]:
#Standard setup
import pandas as pd
import numpy as np
import torch
!pip install git+https://github.com/cmacdonald/spotlight.git@master#egg=spotlight
from spotlight.interactions import Interactions
SEED=20
BPRMF=None

  Cloning https://github.com/cmacdonald/spotlight.git (to revision master) to /tmp/pip-install-4b464kh1/spotlight_50c58705fdef4109b1e4609b00b2a424
  Running command git clone -q https://github.com/cmacdonald/spotlight.git /tmp/pip-install-4b464kh1/spotlight_50c58705fdef4109b1e4609b00b2a424
  Created wheel for spotlight: filename=spotlight-0.1.6-py3-none-any.whl size=34106 sha256=c7827ba95467f8f451abc8ff41574f7345b8622b5f38c37f45750a74b4b24554
  Stored in directory: /tmp/pip-ephem-wheel-cache-olsq7jq7/wheels/1c/2a/31/d187173520bc800643df4e3d1f97dee21d2133ba41085704ed
Successfully built spotlight


## Pre 2. Data Preparation

Let's load the dataset into dataframes.

In [ ]:
#load in the csv files
ratings_df = pd.read_csv("ratings.csv")
books_df = pd.read_csv("books.csv")
to_read_df = pd.read_csv("to_read.csv")
test = pd.read_csv("test.csv")

In [ ]:
#cut down the number of items and users
counts=ratings_df[ratings_df["book_id"] < 2000].groupby(["book_id"]).count().reset_index()
valid_books=counts[counts["user_id"] >= 10][["book_id"]]

books_df = books_df.merge(valid_books, on="book_id")
ratings_df = ratings_df[ratings_df["user_id"] < 2000].merge(valid_books, on="book_id")
to_read_df = to_read_df[to_read_df["user_id"] < 2000].merge(valid_books, on="book_id")
test = test[test["user_id"] < 2000].merge(valid_books, on="book_id")


#stringify the id columns
def str_col(df):
  if "user_id" in df.columns:
    df["user_id"] = "u" + df.user_id.astype(str)
  if "book_id" in df.columns:
    df["book_id"] = "b" + df.book_id.astype(str)

str_col(books_df)
str_col(ratings_df)
str_col(to_read_df)
str_col(test)

Here we construct the Interactions objects from `ratings.csv`, `to_read.csv` and `test.csv`. We manually specify the num_users and num_items parameters to all Interactions objects, in case the test set differs from your training sets.

In [ ]:
from collections import defaultdict
from itertools import count

from spotlight.cross_validation import random_train_test_split

iid_map = defaultdict(count().__next__)


rating_iids = np.array([iid_map[iid] for iid in ratings_df["book_id"].values], dtype = np.int32)
test_iids = np.array([iid_map[iid] for iid in test["book_id"].values], dtype = np.int32)
toread_iids = np.array([iid_map[iid] for iid in to_read_df["book_id"].values], dtype = np.int32)


uid_map = defaultdict(count().__next__)
test_uids = np.array([uid_map[uid] for uid in test["user_id"].values], dtype = np.int32)
rating_uids = np.array([uid_map[uid] for uid in ratings_df["user_id"].values], dtype = np.int32)
toread_uids = np.array([uid_map[iid] for iid in to_read_df["user_id"].values], dtype = np.int32)


uid_rev_map = {v: k for k, v in uid_map.items()}
iid_rev_map = {v: k for k, v in iid_map.items()}


rating_dataset = Interactions(user_ids=rating_uids,
                               item_ids=rating_iids,
                               ratings=ratings_df["rating"].values,
                               num_users=len(uid_rev_map),
                               num_items=len(iid_rev_map))

toread_dataset = Interactions(user_ids=toread_uids,
                               item_ids=toread_iids,
                               num_users=len(uid_rev_map),
                               num_items=len(iid_rev_map))

test_dataset = Interactions(user_ids=test_uids,
                               item_ids=test_iids,
                               num_users=len(uid_rev_map),
                               num_items=len(iid_rev_map))

print(rating_dataset)
print(toread_dataset)
print(test_dataset)

#here we define the validation set
toread_dataset_train, validation = random_train_test_split(toread_dataset, random_state=np.random.RandomState(SEED))

num_items = test_dataset.num_items
num_users = test_dataset.num_users

<Interactions dataset (1999 users x 1826 items x 124762 interactions)>
<Interactions dataset (1999 users x 1826 items x 135615 interactions)>
<Interactions dataset (1999 users x 1826 items x 33917 interactions)>


Finally, this is some utility code that we will use in the exercise.

In [ ]:
def getAuthorTitle(iid):
  bookid = iid_rev_map[iid]
  row = books_df[books_df.book_id == bookid]
  return row.iloc[0]["authors"] + " / " + row.iloc[0]["title"]

print("iid 0: " + getAuthorTitle(0) )

iid 0: Carlos Ruiz Zafón, Lucia Graves / The Shadow of the Wind (The Cemetery of Forgotten Books,  #1)


## Pre 3. Example Code

To evaluate some of your hand-implemented recommender systems (e.g. Q1, Q4), you will need to instantiate objects that match the specification of a Spotlight model, which `mrr_score()` etc. expects.


Here is an example recommender object that returns 0 for each item, regardless of user.

In [ ]:
from spotlight.evaluation import mrr_score, precision_recall_score

class dummymodel:
  
  def __init__(self, numitems):
    self.predictions=np.zeros(numitems)
  
  #uid is the user we are requesting recommendations for;
  #returns an array of scores, one for each item
  def predict(self, uid):
    #this model returns all zeros, regardless of userid
    return( self.predictions )

#lets evaluate how the effeciveness of dummymodel

print(mrr_score(dummymodel(num_items), test_dataset, train=rating_dataset, k=100).mean())
#as expected, a recommendation model that gives 0 scores for all items obtains a MRR score of 0

0.0


In [ ]:
#note that mrr_score() displays a progress bar if you set verbose=True
print(mrr_score(dummymodel(num_items), test_dataset, train=rating_dataset, k=100, verbose=True).mean())


1999it [00:00, 2821.92it/s]

0.0


# Part-A. Combination of Recommendation Models

## Task 1. Explicit & Implicit Matrix Factorisation Models

Create and train three matrix factorisation systems:
 - "EMF": explicit MF, trained on the ratings Interactions object (`rating_dataset`)
 - "IMF": implicit MF, trained on the toread_dataset Interactions object (`toread_dataset_train`)
 - "BPRMF": implicit MF with the BPR loss function (`loss='bpr'`), trained on the toread_dataset Interactions object (`toread_dataset_train`)

Use a variable of the same name for these models, as we will use some of them later (e.g. `BPRMF`).
  
In all cases, you must use the standard initialisation arguments, i.e. 
`n_iter=10, embedding_dim=32, use_cuda=False, random_state=np.random.RandomState(SEED)`.
 
Evaluate each of these models in terms of Mean Reciprocal Rank on the test set. MRR can be obtained using:
```python
mrr_score(X, test_dataset, train=rating_dataset, k=100, verbose=True).mean())
```
where X is an instance of a Spotlight model. Do NOT change the `k` or `train` arguments.

In [ ]:
# Add your solution here
from spotlight.factorization.explicit import ExplicitFactorizationModel
from spotlight.factorization.implicit import ImplicitFactorizationModel
import time  


#a)
EMF = ExplicitFactorizationModel(n_iter=10,
                                    embedding_dim=32, 
                                    use_cuda=False,
                                    random_state=np.random.RandomState(SEED) 
)
current = time.time()
EMF.fit(rating_dataset, verbose=True)
end = time.time()

print("EMF Training took %d seconds \n "% (end - current))

mrr_EMF = mrr_score(EMF, test_dataset, train=rating_dataset, k=100, verbose=True)
print("MRR score for EMF : %f" %(mrr_EMF.mean()))

print("\n------------------------------------------------")

#b)
IMF = ImplicitFactorizationModel( n_iter=10, 
                                    embedding_dim=32, 
                                    use_cuda=False,
                                    random_state=np.random.RandomState(SEED)
)
current = time.time()
IMF.fit(toread_dataset_train, verbose=True)
end = time.time()

print("IMF Training took %d seconds \n" % (end - current))

mrr_IMF = mrr_score(IMF, test_dataset, train=rating_dataset, k=100, verbose=True)
print("MRR score for IMF : %f" %(mrr_IMF.mean()))

print("\n------------------------------------------------")

#c)
BPRMF = ImplicitFactorizationModel( n_iter=10, 
                                    embedding_dim=32, 
                                    use_cuda=False,
                                    random_state=np.random.RandomState(SEED), 
                                    loss='bpr'
)
current = time.time()
BPRMF.fit(toread_dataset_train, verbose=True)
end = time.time()

print("BPRMF Training took %d seconds. \n" % (end - current))

mrr_BPRMF = mrr_score(BPRMF, test_dataset, train=rating_dataset, k=100, verbose=True)
print("MRR score for BPRMF : %f" %(mrr_BPRMF.mean()))

print("\n------------------------------------------------")


Epoch 0: loss 3.8710271667261593
Epoch 1: loss 0.7940810446123607
Epoch 2: loss 0.6382512643200452
Epoch 3: loss 0.5217335281557725
Epoch 4: loss 0.44844855655167926
Epoch 5: loss 0.40543351120880394
Epoch 6: loss 0.3823863151254224
Epoch 7: loss 0.36336620252762664
Epoch 8: loss 0.35137936695799477





0it [00:00, ?it/s]


104it [00:00, 1032.10it/s]

Epoch 9: loss 0.3396692546237199
EMF Training took 11 seconds 
 





203it [00:00, 1018.48it/s]


309it [00:00, 1029.82it/s]


407it [00:00, 1013.17it/s]


515it [00:00, 1029.87it/s]


620it [00:00, 1034.50it/s]


728it [00:00, 1046.91it/s]


837it [00:00, 1059.31it/s]


938it [00:00, 1036.57it/s]


1038it [00:01, 1016.36it/s]


1137it [00:01, 982.89it/s] 


1236it [00:01, 984.56it/s]


1342it [00:01, 1005.01it/s]


1445it [00:01, 1011.22it/s]


1550it [00:01, 1020.55it/s]


1659it [00:01, 1037.30it/s]


1766it [00:01, 1043.62it/s]


1871it [00:01, 1035.49it/s]


1999it [00:01, 1018.58it/s]


MRR score for EMF : 0.058984

------------------------------------------------
Epoch 0: loss 0.7677980539090229
Epoch 1: loss 0.53877861825925
Epoch 2: loss 0.47017199658560305
Epoch 3: loss 0.428322009882837
Epoch 4: loss 0.39839018825090156
Epoch 5: loss 0.368275504770144
Epoch 6: loss 0.3473479778699155
Epoch 7: loss 0.32980164804689166
Epoch 8: loss 0.31870100696413023


...

MRR score for IMF : 0.329932

------------------------------------------------
Epoch 0: loss 0.33895447579616644
Epoch 1: loss 0.19644999289709442
Epoch 2: loss 0.15870640168563938
Epoch 3: loss 0.14147728193059284
Epoch 4: loss 0.132827276100387
Epoch 5: loss 0.12213623321632731
Epoch 6: loss 0.11668406535853755
Epoch 7: loss 0.11047121562626001
Epoch 8: loss 0.10888675406996934





0it [00:00, ?it/s]


107it [00:00, 1057.60it/s]

Epoch 9: loss 0.10400472129782978
BPRMF Training took 17 seconds. 






204it [00:00, 1029.50it/s]


315it [00:00, 1050.22it/s]


423it [00:00, 1058.95it/s]


528it [00:00, 1053.70it/s]


630it [00:00, 1042.37it/s]


729it [00:00, 1021.40it/s]


823it [00:00, 980.32it/s] 


920it [00:00, 975.31it/s]


1027it [00:01, 999.58it/s]


1130it [00:01, 1005.69it/s]


1235it [00:01, 1013.75it/s]


1338it [00:01, 1018.07it/s]


1448it [00:01, 1041.07it/s]


1552it [00:01, 1035.44it/s]


1656it [00:01, 1033.86it/s]


1762it [00:01, 1039.07it/s]


1867it [00:01, 1041.64it/s]


1999it [00:01, 1024.18it/s]

MRR score for BPRMF : 0.407677

------------------------------------------------


## Task 2. Hybrid Model

In this task, you are expected to create new hybrid recommendation models that 
combine the two models in Task 1, namely IMF and BPRMF. 

(a) Linearly combine the *scores* from IMF and BPRMF.  Normalise both input scores into the range 0..1 using [sklearn's minmax_scale() function](
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.minmax_scale.html) before combining them.

(b) Apply a pipelining recommender, where the top 100 items are obtained from IMF and re-ranked using the scores of BPRMF. Items not returned by IMF get a score of 0.

To implement these hybrid models, you should create new classes that abide by the Spotlight model contract (namely, it has a `predict(self, uid)` function that returns a score for *all* items). 

Evaluate each model in terms of MRR. How many users are improved, how many are degraded compared to the BPRMF baseline?

Finally, pass your instantiated model object to the `test_Hybrid_a()` (for (a)) or `test_Hybrid_b()` (for (b)) functions, as appropriate, and record the results in the quiz. For example, if your model for (b) is called `pipeline`, then you would run:
```python
test_Hybrid_b(pipeline)
```

You now have sufficient information to answer the Task 2 quiz questions.

In [ ]:
# len(IMF.predict(0))
# b = (BPRMF.predict(0))

# a = IMF.predict(0)
# a_in = np.argsort(a)[::-1]
# a[23]
# # a[1204](min)
# (np.argmin(a))
# # a[23]
# # a[1204]
# x = a_in[0:100]
# for i in x:
#   print(b[i])

In [ ]:
def test_Hybrid_a(combsumObj):
  for i, u in enumerate([5, 20]):
    print("Hybrid a test case %d" % i)
    print(np.count_nonzero(combsumObj.predict(u) > 1))

def test_Hybrid_b(pipeObj):
  for i, iid in enumerate([3, 0]):
    print("Hybrid b test case %d" % i)
    print(pipeObj.predict(0)[iid])



In [ ]:
# Add your solutions here and evaluate them
from sklearn import preprocessing
from scipy.stats import rankdata


class linearModel:
  
  def __init__(self, num_items):
    self.predictions = np.zeros(num_items)
  
  def predict(self, uid):

    imf_score = IMF.predict(uid)
    bprmf_score = BPRMF.predict(uid)
    norm_IMF_score = preprocessing.minmax_scale(imf_score, feature_range = (0,1))
    norm_BPRMF_score = preprocessing.minmax_scale(bprmf_score, feature_range = (0,1))

    #combsumobj = Scores of sums from each input ranking
    val = norm_IMF_score + norm_BPRMF_score

    return val

linear_model_mrr = mrr_score(linearModel(num_items), test_dataset, train=rating_dataset, k=100)
print("Linear Model MRR score :", linear_model_mrr.mean())

deg = 0
imp = 0

for i in range(0,len(mrr_BPRMF)):
  if mrr_BPRMF[i] < linear_model_mrr[i]:
    imp += 1
  elif mrr_BPRMF[i] > linear_model_mrr[i]:
    deg +=1

print('RR improved by linear :', imp)
print('RR degraded by linear :', deg)


class pipeModel:
  
  def __init__(self, num_items):
    self.predictions = np.zeros(num_items)
  
  def predict(self, uid):

    imf_score = IMF.predict(uid)
    bprmf_score = BPRMF.predict(uid)
    imf_score_sort = np.argsort(imf_score)[::-1][0:100]
    
    imf_reranked = np.zeros(len(bprmf_score))

    for i in imf_score_sort:
      imf_reranked[i] = bprmf_score[i]

    return imf_reranked

pipeline_model_mrr = mrr_score(pipeModel(num_items), test_dataset, train=rating_dataset, k=100)
print("Pipeline Model MRR score :", pipeline_model_mrr.mean())

deg = 0
imp = 0

for i in range(0,len(mrr_BPRMF)):
  if mrr_BPRMF[i] < pipeline_model_mrr[i]:
    imp += 1
  elif mrr_BPRMF[i] > pipeline_model_mrr[i]:
    deg +=1

print('RR improved by pipeline :', imp)
print('RR degraded by pipeline :', deg)

Linear Model MRR score : 0.41010939818893816
RR improved by linear : 736
RR degraded by linear : 745
Pipeline Model MRR score : 0.41511955311399246
RR improved by pipeline : 586
RR degraded by pipeline : 214


In [ ]:
#Now test your hybrid approaches for the quiz

test_Hybrid_a(linearModel(0))
test_Hybrid_b(pipeModel(0))


Hybrid a test case 0
445
Hybrid a test case 1
407
Hybrid b test case 0
22.013418197631836
Hybrid b test case 1
0.0


# Part-B. Analysing Recommendation Models

## Utility methods

Below, we provide a function, `get_top_K(model, uid : int, k : int)` which, when provided with a Spotlight model, will provide the top k predictions for the specified uid. The iids, their scores, and their embeddings are returned. 

In [ ]:
from typing import Sequence, Tuple

def get_top_K(model, uid : int, k : int) -> Tuple[ Sequence[int], Sequence[float],  np.ndarray ] :
  #returns iids, their (normalised) scores in descending order, and item emebddings for the top k predictions of the given uid.

  from sklearn.preprocessing import minmax_scale

  from scipy.stats import rankdata
  # get scores from model
  scores = model.predict(uid)

  # map scores into rank 0..1 over the entire item space
  scores = minmax_scale(scores)

  #compute their ranks  
  ranks = rankdata(-scores)
  
  # get and filter iids, scores and embeddings
  rtr_scores = scores[ranks <= k]
  rtr_iids = np.argwhere(ranks <= k).flatten()
  if hasattr(model, '_net'):
    embs = model._net.item_embeddings.weight[rtr_iids]
  else:
    # not a model that has any embeddings
    embs = np.zeros([k,1])
  
  # identify correct ordering using numpy.argsort()
  ordering = (-1*rtr_scores).argsort()
  
  #return iids, scores and their embeddings in descending order of score
  return rtr_iids[ordering], rtr_scores[ordering], embs[ordering]

if BPRMF is not None:
  iids, scores, embs = get_top_K(BPRMF, 0, 10)
  print("Returned iids: %s" % str(iids))
  print("Returned scores: %s" % str(scores))
  print("Returned embeddings: %s" % str(embs))
else:
  print("You need to define BPRMF in Task 1")



Returned iids: [ 23 108  21  33   9  81  52 254  16   3]
Returned scores: [1.         0.9895131  0.9848315  0.92250896 0.9070817  0.90654314
 0.9005319  0.89310133 0.88378096 0.8836929 ]
Returned embeddings: tensor([[-0.0453,  1.3716, -0.8307, -1.2616,  1.6700,  1.0161,  1.1168,  2.3530,
         -1.2027,  0.8522, -1.0941, -0.6865, -0.5725, -2.0335, -1.2591,  0.6154,
         -0.1374, -1.6868, -1.8615, -0.7514,  1.9909, -0.3909,  1.9239,  1.3293,
         -1.2834, -0.4520,  1.1338,  0.3467,  2.5169, -2.1587,  1.2310,  1.1670],
        [ 0.1239,  1.1004,  0.0531, -1.1045,  1.9932,  1.5049,  1.0011,  1.9734,
         -1.6322, -0.8913, -0.6372,  0.7721, -1.1422, -2.2424, -1.1936, -0.5770,
          0.0762, -1.0283, -1.2807, -2.0889,  2.8154, -0.9600, -0.1419,  0.8408,
         -1.6067, -1.2905,  1.9169,  1.3988,  1.8646, -2.2028,  0.5365,  0.2022],
        [ 0.3845,  0.8188, -0.1892, -1.1793,  2.1731,  0.6669,  1.1271,  1.4538,
         -1.2173, -0.5447, -1.6713,  0.5249, -0.6132, -3.1082

## Task 3. Evaluation of Non-personalised Models
Implement the following four (non-personalised) baselines for ranking books based on their statistics:
 - Average rating, obtained from ratings_df, `ratings` column
 - Number of ratings, obtained from books_df (column `ratings_count`)
 - Number of 5* ratings, obtained from books_df (column `ratings_5`)
 - Fraction of 5* ratings, calculated from the two sources of evidence above, i.e (columns  `ratings_5` and `ratings_count`).

Evaluate these in terms of MRR using the provided test data. You may use the StaticModel class below. 

Hints: 
 - As in Exercise 2, the order of items returned by predict() is _critical_. You may wish to refer to iid_map.
 - For all models, you need to ensure that your values are not cast to ints. If you are extracting values from a Pandas series, it is advised to use [.astype(np.float32)](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.astype.html).


In [ ]:
class StaticModel:
  
  def __init__(self, staticscores):
    self.numitems = len(staticscores)
    #print(self.numitems)
    assert isinstance(staticscores, np.ndarray), "Expected a numpy array"
    assert staticscores.dtype == np.float32 or staticscores.dtype == np.float64, "Expected a numpy array of floats"
    self.staticscores = staticscores
  
  def predict(self, uid):
    #this model returns the same scores for each user    
    return self.staticscores

In [ ]:
#len(ratings_df['book_id'].unique())
# ratings_df['rating'].min()
# ratings_df['rating'].max()

In [ ]:
# books_df.head(2)
# books_df['ratings_count'].min()
# books_df['ratings_count'].max()

In [ ]:
# iid_map.get('b123')

In [ ]:
# books_df
# books_df['book_id'][1]
# for i,r in books_df.iterrows():
#books_df.dtypes


In [ ]:
# Add your solution here

#a)
a = np.zeros(num_items)
for i, row in ratings_df.groupby(by=['book_id']).mean().iterrows():
  a[iid_map[i]] = row['rating']

model_a = StaticModel(a.astype(np.float32))
mrr_a = mrr_score(model_a, test_dataset,train = rating_dataset, k=100, verbose=True)
print('\n MRR Score for model a(average rating) = ', mrr_a.mean())

#b)
temp1 = np.zeros(num_items)
for i, row in books_df.iterrows():
  temp1[iid_map[books_df['book_id'][i]]] = row['ratings_count']
b = ( temp1 / np.max(temp1) ) * 5
model_b = StaticModel(b.astype(np.float32))

mrr_b = mrr_score(model_b, test_dataset,train = rating_dataset , k=100, verbose=True)
print('\n MRR Score for model b(number of ratings) = ', mrr_b.mean())

#c)
temp2 = np.zeros(num_items)
for i, row in books_df.iterrows():
  temp2[iid_map[books_df['book_id'][i]]] = row['ratings_5']
c = ( temp2 / np.max(temp2) ) * 5
model_c = StaticModel(c.astype(np.float32))

mrr_c = mrr_score(model_c, test_dataset,train = rating_dataset , k=100, verbose=True)
print('\n MRR Score for model c(number of 5 rating) = ', mrr_c.mean())

#d)
temp3 = np.zeros(num_items)
for i, row in books_df.iterrows(): 
  temp3[iid_map[books_df['book_id'][i]]] = (row['ratings_5'] / row['ratings_count'])
d = ( temp3 / np.max(temp3) ) * 5
model_d = StaticModel(d.astype(np.float32))

mrr_d = mrr_score(model_d, test_dataset,train = rating_dataset , k=100, verbose=True)
print('\n MRR Score for model d(fraction of 5 rating) = ', mrr_d.mean())





0it [00:00, ?it/s]


215it [00:00, 2148.35it/s]


421it [00:00, 2119.45it/s]


601it [00:00, 2009.49it/s]


790it [00:00, 1972.02it/s]


992it [00:00, 1983.50it/s]


1201it [00:00, 2012.53it/s]


1391it [00:00, 1975.35it/s]


1591it [00:00, 1980.68it/s]


1790it [00:00, 1982.42it/s]


1999it [00:01, 1951.95it/s]



0it [00:00, ?it/s]


 MRR Score for model a(average rating) =  0.015052024168984034





198it [00:00, 1979.76it/s]


419it [00:00, 2042.43it/s]


614it [00:00, 2011.91it/s]


823it [00:00, 2034.20it/s]


1028it [00:00, 2037.87it/s]


1242it [00:00, 2065.75it/s]


1465it [00:00, 2110.22it/s]


1687it [00:00, 2139.73it/s]


1999it [00:00, 2090.20it/s]



0it [00:00, ?it/s]


 MRR Score for model b(number of ratings) =  0.2396001188245477





221it [00:00, 2204.01it/s]


426it [00:00, 2154.23it/s]


645it [00:00, 2163.76it/s]


817it [00:00, 2005.58it/s]


1019it [00:00, 2009.13it/s]


1230it [00:00, 2036.93it/s]


1438it [00:00, 2047.75it/s]


1628it [00:00, 1998.61it/s]


1999it [00:00, 2025.29it/s]



0it [00:00, ?it/s]


 MRR Score for model c(number of 5 rating) =  0.2409670879930144





221it [00:00, 2204.96it/s]


420it [00:00, 2135.57it/s]


639it [00:00, 2149.67it/s]


831it [00:00, 2074.24it/s]


1039it [00:00, 2074.12it/s]


1255it [00:00, 2098.04it/s]


1451it [00:00, 2052.65it/s]


1652it [00:00, 2038.30it/s]


1999it [00:00, 2039.02it/s]


 MRR Score for model d(fraction of 5 rating) =  0.03415267465103555


In [ ]:
#d.dtype

## Task 4. Qualiatively Examining Recommendations

From now on, we will consider the `BPRMF` model.

In Recommender Systems, the ground truth (i.e. our list of books that the user has added to their "to_read" shelf) can be very incomplete. For instance, this can be because the user is not aware of the book yet.

For this reason, it is important to "eyeball" the recommendations, to understand what the system is surfacing, and whether the recommendations make sense. In this way, we understand if the recommendations are reasonable, even if they are for books that the user has not actually read according to the test dataset.

First, write a function, which given a uid (int), prints the *title and authors* of:
 - (a) the books that the user has previously shelved (c.f. `toread_dataset`)
 - (b) the books that the user will read in the future (c.f. `test_dataset`)
 - (c) the top 10 books that the user were recommended by `BPRMF` - you can make use of `get_top_K()`.

You can use the previously defined `getAuthorTitle()` function in your solution.
You will also want to compare books in (c) with those in (a) and (b).

Then, we will examine two specific users, namely uid 1805 (u336) and uid 179 (user u1331), to analyse if their recommendations make sense. Refer to the Task 4 quiz questions.


In [ ]:
toread_dataset

<Interactions dataset (1999 users x 1826 items x 135615 interactions)>

In [ ]:

# for i in toread_dataset.item_ids[toread_dataset.user_ids == 0]:
#   print(i)
# getAuthorTitle(56)

In [ ]:
# Add your solution here
recommended = []
future_read = []
previously_shelved = []

def get_title_author_4(uid):
  print('\nUID :', uid)
  
  
  print('The books that the user has previously shelved : ')
  for iid in toread_dataset.item_ids[toread_dataset.user_ids == uid]:
    authorTitle = getAuthorTitle(iid)
    previously_shelved.append(authorTitle)
    print(iid,':',authorTitle)

  print('(Totat count = %d)'%(len(previously_shelved)))
  # print(previously_shelved)

  
  print('\n The books that the user will read in the future: ')
  for iid in test_dataset.item_ids[test_dataset.user_ids == uid]:
    authorTitle = getAuthorTitle(iid)
    future_read.append(authorTitle)
    print(iid,':',authorTitle)
  print('(Totat count = %d)'%(len(future_read)))
  #print(future_read)

  
  k = 10
  print('\n The top 10 books that the user were recommended:') 
  recommended_iids, scores, embds = get_top_K(BPRMF, uid , k )
  for iid in recommended_iids:
    authorTitle = getAuthorTitle(iid)
    recommended.append(authorTitle)
    print(iid,':',authorTitle)
  print(' (Totat count = %d)'%(len(recommended)))
  # print(recommended)

In [ ]:
recommended = []
future_read = []
previously_shelved = []
get_title_author_4(1805)


UID : 1805
The books that the user has previously shelved : 
1348 : Stieg Larsson, Reg Keeland / The Girl Who Kicked the Hornet's Nest (Millennium, #3)
1390 : Suzanne Collins / Mockingjay (The Hunger Games, #3)
727 : Dennis Lehane / Shutter Island
1342 : Suzanne Collins / Catching Fire (The Hunger Games, #2)
1771 : Paula Hawkins / The Girl on the Train
743 : Robert Ludlum / The Bourne Supremacy (Jason Bourne, #2)
92 : John Grisham / The Client
566 : Thomas Harris / The Silence of the Lambs  (Hannibal Lecter, #2)
243 : Daphne du Maurier, Sally Beauman / Rebecca
40 : Robert Ludlum / The Bourne Identity (Jason Bourne, #1)
1687 : Robert Galbraith, J.K. Rowling / The Cuckoo's Calling (Cormoran Strike, #1)
613 : Stephen King / Misery
440 : Michael Crichton / Jurassic Park (Jurassic Park, #1)
812 : Robert Ludlum / The Bourne Ultimatum (Jason Bourne, #3)
167 : Stephen King, Bernie Wrightson / The Stand
457 : Michael Crichton / The Andromeda Strain
188 : Thomas Harris / Red Dragon (Hannibal Le

In [ ]:
round(mrr_BPRMF[1805],4)

0.0556

In [ ]:
a = [x for x in previously_shelved if x in recommended]
print(a)

['Suzanne Collins / Mockingjay (The Hunger Games, #3)']


In [ ]:
a = [x for x in future_read if x in recommended]
print(len(a))


0


In [ ]:
get_title_author_4(179)


UID : 179
The books that the user has previously shelved : 
75 : Dan Brown / Angels & Demons  (Robert Langdon, #1)
1270 : Suzanne Collins / The Hunger Games (The Hunger Games, #1)
244 : Antoine de Saint-Exupéry, Richard Howard, Dom Marcos Barbosa, Melina Karakosta / The Little Prince
95 : Truman Capote / Breakfast at Tiffany's
16 : Dan Brown / The Da Vinci Code (Robert Langdon, #2)
926 : Laura Ingalls Wilder, Garth Williams / Little House on the Prairie (Little House, #2)
147 : Milan Kundera, Michael Henry Heim / The Unbearable Lightness of Being
1342 : Suzanne Collins / Catching Fire (The Hunger Games, #2)
92 : John Grisham / The Client
261 : J.R.R. Tolkien / The Lord of the Rings (The Lord of the Rings, #1-3)
343 : J.R.R. Tolkien / The Hobbit
180 : Margaret Mitchell / Gone with the Wind
594 : Neil Gaiman / Stardust
1021 : Laura Ingalls Wilder, Garth Williams / Little House in the Big Woods (Little House, #1)
655 : Pearl S. Buck / The Good Earth (House of Earth, #1)
778 : Dan Brown /

# Part-C. Diversity of Recommendations

This part of the exercise is concerned with diversification, as covered in Lecture 11.

## Task 5. Measuring Intra-List Diversity


For the BPR implicit factorisation model, implement the Intra-list diversity measure (see Lecture 11) of the top 5 scored items based on their item embeddings in the `BPRMF` model. 

Implement your ILD as a function with the specification:
```python
def measure_ild(top_books : Sequence[int], K : int=5) -> float
```
where:
 - `top_books` is a list or a Numpy array of iids that have been returned for a particular user. For instance, it can be obtained from `get_top_K()`.
 - `K` is the number of top-ranked items to consider from `top_books`. 
 - Your implementation should use the item emebddings stored in the `BPRMF` model.

Calculate the ILD (with k=5). Using your code for Task 4, identify the books previously shelved and recommended for the specific users requested in the quiz, and use these to analyse the recommendations.

Hints:
 - As can be seen in `get_top_K()`, item embeddings can be obtained from `BPRMF._net.item_embeddings.weight[iid]`.
 - For obtaining the cosine similarity of PyTorch tensors, use `nn.functional.cosine_similarity(, , axis=0)`.


In [ ]:
# import numpy as np
# arr = np.array([1,2,3,4,5])
# cosine_similarities = []
# l = len(arr)
# d = []
# for i in range(0,l):
#   for j in range(i+1,l):
#      d.append(1-(arr[i] - arr[j]))
# print(np.sum(d)) 
# fin = 2 /(l * (l-1))   
    

In [ ]:
# uids,scores,embd = get_top_K(BPRMF,0,5)
# embd
# # BPRMF._net.item_embeddings.weight[23]

In [ ]:
# for i in range(0, 5):
#     for j in range(0, 5):
#       print(i,j)

In [ ]:
# first = BPRMF._net.item_embeddings.weight[1]
# second = BPRMF._net.item_embeddings.weight[0]
# nn.functional.cosine_similarity(first ,second , axis=0)

In [ ]:
# Add your solution here
import torch.nn as nn
import numpy as np

def measure_ild(top_books : Sequence[int], K : int=5) -> float:
  ILD = 0.0
  iids_len = len(top_books)
  d = []

  for i in range(0, K):
    for j in range(i+1, K):
      if(i == j):
        continue
      first = BPRMF._net.item_embeddings.weight[top_books[i]]
      second = BPRMF._net.item_embeddings.weight[top_books[j]]
      cosine_similarities = nn.functional.cosine_similarity(first ,second , axis=0)
      d.append(1 - cosine_similarities)
  
  ILD = (2 * np.sum(d)/ ( iids_len * (iids_len - 1))) 
  ILD = ILD.double().item()
  return ILD

iids,scores,embd = get_top_K(BPRMF , 0, 5)
ILD = measure_ild(iids)
print(ILD)

0.20721983909606934


In [ ]:
iids,scores,embd = get_top_K(BPRMF , 1805, 5)
ILD = measure_ild(iids)
print(ILD)
for i in iids:
  print(getAuthorTitle(i))

0.7484899163246155
Suzanne Collins / The Hunger Games (The Hunger Games, #1)
Dan Brown / The Da Vinci Code (Robert Langdon, #2)
Dan Brown / The Lost Symbol (Robert Langdon, #3)
Michael Crichton / Disclosure
George R.R. Martin / A Clash of Kings  (A Song of Ice and Fire, #2)


In [ ]:
iids,scores,embd = get_top_K(BPRMF , 179, 5)
ILD = measure_ild(iids)
print(round(ILD,4))
for i in iids:
  print(getAuthorTitle(i))

0.2787
John Grisham / The Partner
John Grisham / The Pelican Brief
John Grisham / The Client
John Grisham / The Brethren
John Grisham / The Street Lawyer


## Task 6. Implement MMR Diversification 

Develop an Maximal Marginal Relevance (M**M**R) diversification technique, to re-rank the top-ranked recommendations for a given user.

Your function should adhere to the specification as follows:
```python
def mmr(iids : Sequence[int], scores : Sequence[float], embs : np.ndarray, alpha : float) -> Sequence[int]:
```

where iids is a list of iids, scores are their corresponding scores (in descending order), embs is their embeddings, and alpha controls the diversification tradeoff. The function returns a re-ordering of iids. As in previous Exercises, type hints are provided for clarity; a Sequence can be a list or numpy array. 

Hints:
 - As above, for obtaining the cosine similarity of PyTorch tensors, use nn.functional.cosine_similarity(, , axis=0).

To use your `mmr()` function, provide it with the outputs of `get_top_K()`. For example, to obtain an MMR reordering of the top 10 predictions of uid 0, we can run:
```
mmr( *get_top_K(bprmodel, 0, 10), 0.5)
```

Thereafter, we provide test cases for your MMR implementation, which you  should report in the quiz. We also ask for the ILD values before and after the application of MMR.


In [ ]:
main_list = np.setdiff1d(R,S)

In [ ]:
# sim = np.array([[1,0.11,0.23,0.76,0.25],[0,1,0.29,0.57,0.51],[0,0,1,0.04,0.2,],[0,0,0,1,0.33],[0,0,0,0,1]])
# scores = [0.91,0.9,0.5,0.06,0.63]
# R = [0,1,2,3,4]
# lam1 = 0.5
# lam2 = 1 - lam1
# s = []
# final_scores =[]
# x = 0

# while len(R) > 0:
   
#     print('\n iteration:', x )
#     mrr_score = 0
#     iid = ''
#     # score_list = [0]
#     for i in R:
#         similarity = 0
#         temp = [0]

#         for j in s:    
#           temp.append(sim[j][i])

#         similarity = np.max(temp)
#         mrr_temp = (lam1  * scores[i]) - (lam2 * similarity)
        
#         if mrr_temp > mrr_score:
#             mrr_score = mrr_temp
#             iid = i
#     if iid == '':
#         iid = i

#     temp = R.copy()
#     if iid in temp: temp.remove(iid)
#     R = temp.copy()
    
#     s.append(iid)
#     final_scores.append(mrr_score)
#     x += 1   
   
# s
# final_scores
# s

In [ ]:
from typing import Sequence
def mmr(iids : Sequence[int], scores : Sequence[float], embs : np.ndarray, alpha : float) -> Sequence[int]:

  assert len(iids) == len(scores)
  assert len(iids) == embs.shape[0]
  assert len(embs.size()) == 2

  R = iids
  lam1 = alpha
  lam2 = 1 - lam1
  s = []
  final_scores =[]
  # print(R)
  # print(scores)
  while len(R) > 0:
    mrr_score = 0
    iid = ''
    for i in R:
        similarity = 0
        temp = [0]
        for j in s:  
          # print(i)
          # first = BPRMF._net.item_embeddings.weight[iids[iids.index(i)]]
          # second = BPRMF._net.item_embeddings.weight[iids[iids.index(j)]]
          first = embs[iids.index(i)]
          second = embs[iids.index(j)]
          cosine_similarities = nn.functional.cosine_similarity(second ,first , axis=0)
          temp.append(cosine_similarities)
        similarity = np.max(temp)
        # print(i)
        mrr_temp = (lam1  * scores[iids.index(i)]) - (lam2 * similarity)
        if mrr_temp > mrr_score:
            mrr_score = mrr_temp
            iid = i
    if iid == '':
        iid = i
   
    R_copy = R.copy()
    if iid in R_copy: R_copy.remove(iid)
    R = R_copy.copy()
   
    
    s.append(iid)
    final_scores.append(mrr_score)
  
  # print('reorderd',s)
  rtr_iids = s
  
  #input your solution here returns a re-ordering of iids, such that the first ranked item is first in the list

  return rtr_iids

In [ ]:
def run_MMR_testcases(mmrfn):
  example_embeddings1 = torch.tensor([[1.0,1.0],[1.0,1.0],[0,1.0],[0.1, 1.0]])
  example_embeddings2 = torch.tensor([[1.0,1.0],[1.0,1.0],[0.02,1.0],[0.01,1.0]])
  print("Testcase 0 : %s" % mmrfn([1,2,3,4], [0.5, 0.5, 0.5, 0.5],  example_embeddings1, 0.5)[0] )
  print("Testcase 1 : %s" % mmrfn([1,2,3,4], [0.5, 0.5, 0.5, 0.5],  example_embeddings1, 0.5)[1] )
  print("Testcase 2 : %s" % mmrfn([1,2,3,4], [4, 3, 2, 1],  example_embeddings1, 1)[1] )
  print("Testcase 3 : %s" % mmrfn([1,2,3,4], [0.99, 0.98, 0.97, 0.001],  example_embeddings2, 0.001)[1] )
  print("Testcase 4 : %s" % mmrfn([1,2,3,4], [0.99, 0.98, 0.97, 0.001],  example_embeddings2, 0.5)[1] )

run_MMR_testcases(mmr)

Testcase 0 : 1
Testcase 1 : 4
Testcase 2 : 2
Testcase 3 : 4
Testcase 4 : 3


In [ ]:
# l = np.array([0,2,4,5,6])
# # l.indexOf(2)
# a,b,c = get_top_K(BPRMF, 179, 10)
# type(a)

In [ ]:
# # a,b,c = get_top_K(BPRMF, 179, 10)
# print('c',c[2],a[2])
# a1 = mmr( a.tolist(),b,c, 0.5)
# # print(a1)

Now we can analyse the impact of our MMR implementation. Let's consider again uid 179 (user u1331). 

Apply MMR on the top 10 results obtained from the BPRMF model using `get_top_K()`, with an alpha value of 0.5. The following code should help:
```python
mmr( *get_top_K(bprmodel, 179, 10), 0.5)
```

Finally, anayse the returned books. Calculate the ILD (with `k=5`), and examine the authors and titles (using `getAuthorTitle()`). 

Now answer the questions in Task 6 of the Moodle quiz.


In [ ]:
iids,b,c = get_top_K(BPRMF, 179, 10)
print(iids)
ILD = measure_ild(iids)
print(round(ILD,4))
iids_reranked = a1 = mmr( iids.tolist(), b, c, 0.5)
print(iids_reranked)
for iid in iids_reranked:
  print(getAuthorTitle(iid))
ILD = measure_ild(iids_reranked[0:5])
print(round(ILD,4))

[ 89 391  92 906  88  86  62  91   9 934]
0.0619
[89, 9, 88, 391, 906, 62, 934, 86, 92, 91]
John Grisham / The Partner
J.K. Rowling, Mary GrandPré / Harry Potter and the Sorcerer's Stone (Harry Potter, #1)
John Grisham / The Street Lawyer
John Grisham / The Pelican Brief
John Grisham / The Brethren
John Grisham / The Rainmaker
John Grisham / The Runaway Jury
John Grisham / The Broker
John Grisham / The Client
John Grisham / The King of Torts
0.5566


# Task 7

This task is not a practical task - instead there are questions that tests your understanding of some related content of the course in the quiz.

# End of Exercise

As part of your submission, you should complete the Exercise 3 quiz on Moodle.
You will need to upload your notebook, complete with the **results** of executing the code.